In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [41]:
##Question1

In [4]:
r = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

In [5]:
soup = BeautifulSoup(r,'lxml')
table = soup.table

In [6]:
table_rows = table.find_all('tr')

In [7]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)

In [8]:
df2 = []
for tr in table_rows:
    if(tr.text[6:18] != 'Not assigned'):
        if(tr.text[20:32]  == 'Not assigned'):
            tr.text[20:32] = tr.text[6:18]
        else:
         td = tr.find_all('td')
         row = [tr.text for tr in td]
         df2.append(row)

df2 = pd.DataFrame(df2, columns=["Postal Code", "Borough", "Neighbourhood"])

In [9]:
df2.replace(r'\s+|\\n', '', regex=True, inplace=True)
df2

,Postal Code,Borough,Neighbourhood
0,None,None,None
1,M3A,NorthYork,Parkwoods
2,M4A,NorthYork,VictoriaVillage
3,M5A,DowntownToronto,"RegentPark,Harbourfront"
4,M6A,NorthYork,"LawrenceManor,LawrenceHeights"
...,...,...,...
99,M8X,Etobicoke,"TheKingsway,MontgomeryRoad,OldMillNorth"
100,M4Y,DowntownToronto,ChurchandWellesley
101,M7Y,EastToronto,"BusinessreplymailProcessingCentre,SouthCentral..."
102,M8Y,Etobicoke,"OldMillSouth,King'sMillPark,Sunnylea,HumberBay..."


In [10]:
df=df2.groupby('Postal Code')['Neighbourhood'].apply(lambda x: "%s" % ', '.join(x))
df=df.reset_index(drop=False)
df.rename(columns={'Neighbourhood':'Neighborhood_joined'},inplace=True)

In [11]:
dfm = pd.merge(df2, df, on='Postal Code')
dfm

,Postal Code,Borough,Neighbourhood,Neighborhood_joined
0,M3A,NorthYork,Parkwoods,Parkwoods
1,M4A,NorthYork,VictoriaVillage,VictoriaVillage
2,M5A,DowntownToronto,"RegentPark,Harbourfront","RegentPark,Harbourfront"
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights","LawrenceManor,LawrenceHeights"
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment","Queen'sPark,OntarioProvincialGovernment"
...,...,...,...,...
98,M8X,Etobicoke,"TheKingsway,MontgomeryRoad,OldMillNorth","TheKingsway,MontgomeryRoad,OldMillNorth"
99,M4Y,DowntownToronto,ChurchandWellesley,ChurchandWellesley
100,M7Y,EastToronto,"BusinessreplymailProcessingCentre,SouthCentral...","BusinessreplymailProcessingCentre,SouthCentral..."
101,M8Y,Etobicoke,"OldMillSouth,King'sMillPark,Sunnylea,HumberBay...","OldMillSouth,King'sMillPark,Sunnylea,HumberBay..."


In [12]:
dfm.drop_duplicates(inplace=True)

In [13]:
dfm.drop(['Neighbourhood'],axis=1,inplace=True)

In [14]:
dfm.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [16]:
dfm.shape

(103, 3)

In [17]:
dfm

,Postal Code,Borough,Neighborhood
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,VictoriaVillage
2,M5A,DowntownToronto,"RegentPark,Harbourfront"
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights"
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment"
...,...,...,...
98,M8X,Etobicoke,"TheKingsway,MontgomeryRoad,OldMillNorth"
99,M4Y,DowntownToronto,ChurchandWellesley
100,M7Y,EastToronto,"BusinessreplymailProcessingCentre,SouthCentral..."
101,M8Y,Etobicoke,"OldMillSouth,King'sMillPark,Sunnylea,HumberBay..."


In [18]:
#Q2

In [19]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')

In [20]:
geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [21]:
geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
dfm.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [23]:
merged = pd.merge(geo,dfm, on = 'PostalCode')

In [24]:
merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern,Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"RougeHill,PortUnion,HighlandCreek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood,Morningside,WestHill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [25]:
dff = merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
dff.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,WestHill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [26]:
#Question 3

In [27]:
CLIENT_ID = '2SRCIYFVZUZ2WXSQPQ3KQYZJJZNUV3DG2O5G3X2P3UQEII4C' # your Foursquare ID
CLIENT_SECRET = '4ANKWMIG4E3KPIE2ZNZZ3SGKIAL0AN23LOLMD3HSDMTF2ZZG' # your Foursquare Secret
VERSION = '20200811'
radius=500
LIMIT=100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2SRCIYFVZUZ2WXSQPQ3KQYZJJZNUV3DG2O5G3X2P3UQEII4C
CLIENT_SECRET:4ANKWMIG4E3KPIE2ZNZZ3SGKIAL0AN23LOLMD3HSDMTF2ZZG


In [28]:
dffto=dff[dff['Borough'].str.contains("Toronto")]
dffto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,EastToronto,TheBeaches,43.676357,-79.293031
41,M4K,EastToronto,"TheDanforthWest,Riverdale",43.679557,-79.352188
42,M4L,EastToronto,"IndiaBazaar,TheBeachesWest",43.668999,-79.315572
43,M4M,EastToronto,StudioDistrict,43.659526,-79.340923
44,M4N,CentralToronto,LawrencePark,43.728020,-79.388790


In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
import requests
dfftov = getNearbyVenues(names=dffto['Neighborhood'],
                                   latitudes=dffto['Latitude'],
                                   longitudes=dffto['Longitude']
                                  )

TheBeaches
TheDanforthWest,Riverdale
IndiaBazaar,TheBeachesWest
StudioDistrict
LawrencePark
DavisvilleNorth
NorthTorontoWest,LawrencePark
Davisville
MoorePark,SummerhillEast
SummerhillWest,Rathnelly,SouthHill,ForestHillSE,DeerPark
Rosedale
St.JamesTown,Cabbagetown
ChurchandWellesley
RegentPark,Harbourfront
GardenDistrict,Ryerson
St.JamesTown
BerczyPark
CentralBayStreet
Richmond,Adelaide,King
HarbourfrontEast,UnionStation,TorontoIslands
TorontoDominionCentre,DesignExchange
CommerceCourt,VictoriaHotel
Roselawn
ForestHillNorth&West,ForestHillRoadPark
TheAnnex,NorthMidtown,Yorkville
UniversityofToronto,Harbord
KensingtonMarket,Chinatown,GrangePark
CNTower,KingandSpadina,RailwayLands,HarbourfrontWest,BathurstQuay,SouthNiagara,Islandairport
StnAPOBoxes
FirstCanadianPlace,Undergroundcity
Christie
Dufferin,DovercourtVillage
LittlePortugal,Trinity
Brockton,ParkdaleVillage,ExhibitionPlace
HighPark,TheJunctionSouth
Parkdale,Roncesvalles
Runnymede,Swansea
Queen'sPark,OntarioProvincialGovernment
Bu

In [31]:
dfftov.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,TheBeaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,TheBeaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,TheBeaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,TheBeaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"TheDanforthWest,Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [32]:
dfftov.head().groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
TheBeaches,4,4,4,4,4,4
"TheDanforthWest,Riverdale",1,1,1,1,1,1


In [33]:
# one hot encoding
dfftovonehot = pd.get_dummies(dfftov[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfftovonehot['Neighborhood'] = dfftov['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfftovonehot.columns[-1]] + list(dfftovonehot.columns[:-1])
dfftovonehot = dfftovonehot[fixed_columns]

dfftovonehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
dfftovonehot.shape

(1640, 238)

In [35]:
dff_grouped = dfftovonehot.groupby('Neighborhood').mean().reset_index()
dff_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,BerczyPark,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.016667,0.000000,0.000000,0.000000,0.00
1,"Brockton,ParkdaleVillage,ExhibitionPlace",0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,"BusinessreplymailProcessingCentre,SouthCentral...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,"CNTower,KingandSpadina,RailwayLands,Harbourfro...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.058824,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,CentralBayStreet,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.015625,0.000000,0.000000,0.015625,0.00
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,ChurchandWellesley,0.025974,0.012987,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,...,0.012987,0.012987,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,"CommerceCourt,VictoriaHotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.00
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.029412,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,DavisvilleNorth,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
# set number of clusters
from sklearn.cluster import KMeans
kclusters = 5

dff_grouped_clustering = dff_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dff_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
dff_venues_sorted = pd.DataFrame(columns=columns)
dff_venues_sorted['Neighborhood'] = dff_grouped['Neighborhood']

for ind in np.arange(dff_grouped.shape[0]):
    dff_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dff_grouped.iloc[ind, :], num_top_venues)

In [39]:
# add clustering labels
dff_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dff_merged = dffto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dff_merged = dff_merged.join(dff_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dff_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,EastToronto,TheBeaches,43.676357,-79.293031,3,Pub,Trail,Health Food Store,Women's Store,Discount Store,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
41,M4K,EastToronto,"TheDanforthWest,Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
42,M4L,EastToronto,"IndiaBazaar,TheBeachesWest",43.668999,-79.315572,1,Sandwich Place,Pizza Place,Gym,Food & Drink Shop,Brewery,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Steakhouse
43,M4M,EastToronto,StudioDistrict,43.659526,-79.340923,1,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Clothing Store
44,M4N,CentralToronto,LawrencePark,43.728020,-79.388790,4,Park,Dim Sum Restaurant,Bus Line,Swim School,Women's Store,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [40]:
import folium
from geopy.geocoders import Nominatim 
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dff_merged['Latitude'], dff_merged['Longitude'], dff_merged['Neighborhood'], dff_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


# 